In [3]:
import json
from agents import Agent, Runner, TResponseInputItem

In [4]:
import os

MODEL = os.getenv("OPENAI_MODEL")

In [5]:
primary_agent = Agent(
    name="Fitness Assistant",
    instructions="""
    You are a fitness assistant that helps understand a user, their fitness profile, 
    their fitness profile, and their preferences and goals to provide personalized
    recommendations and guidance
    """,
    model=MODEL
)

In [6]:
result = await Runner.run(
    starting_agent=primary_agent,
    input="Generate a fitness plan for a 150lb male who wants to go the gym 3x a week. Focus on cardio and endurance"
)

print(f"Fitness Assistant: {result.final_output}")

Fitness Assistant: Here's a personalized fitness plan for a 150 lb male focusing on cardio and endurance, tailored for a gym routine three times a week.

### Weekly Fitness Plan

#### Goals:
- Improve cardiovascular fitness
- Build endurance
- Enhance overall fitness levels

### Schedule:

**Day 1: Cardio & Core**
- **Warm-up**: 5-10 minutes of light jogging or brisk walking
- **Main Workout**:
  - Treadmill: 20 minutes (alternating between jogging and sprinting; 1 minute sprint, 2 minutes jog)
  - Cycling: 20 minutes (steady pace with moderate resistance)
- **Core Routine** (2-3 sets):
  - Planks: 30 seconds to 1 minute
  - Bicycle Crunches: 15-20 reps
  - Russian Twists: 15-20 reps each side
- **Cool Down**: 5-10 minutes of stretching

---

**Day 2: Interval Training**
- **Warm-up**: 5-10 minutes of dynamic stretching
- **Main Workout**:
  - Rowing Machine: 10 minutes at moderate pace
  - HIIT Circuit (3 rounds, 30 seconds on, 30 seconds off):
    - Jumping Jacks
    - Burpees
    - 

In [9]:
async def start_chat(primary_agent: Agent, chat: list[TResponseInputItem]):
    
    print("Chat Started")
    print("================")
    
    while True:
        user_input = input("You: ")
        
        if user_input.lower() == "exit":
            print("Ending Chat ...")
            break
        
        print(f"User: {user_input}")
        
        chat.append({
            "content": user_input,
            "role": "user",
            "type": "message"
        })
        
        result = await Runner.run(
            starting_agent=primary_agent,
            input=chat
        )
        
        chat.clear()
        chat.extend(result.to_input_list())
        
        print(f"Assistant: {result.final_output}")

In [10]:
chat = []
await start_chat(primary_agent, chat)

Chat Started
User: HI
Assistant: Hello! How can I assist you with your fitness journey today?
User: I need a gym plan to lose weight
Assistant: Great! To create a personalized gym plan for weight loss, I need a bit more information:

1. **Current Fitness Level**: Are you a beginner, intermediate, or advanced?
2. **Frequency**: How many days a week can you commit to working out?
3. **Duration**: How much time can you dedicate to each workout session?
4. **Equipment**: Do you have access to specific gym equipment?
5. **Any Limitations**: Do you have any injuries or health concerns?
6. **Preferences**: Do you prefer strength training, cardio, or a mix of both?

Once I have this information, I can provide a tailored plan for you!
Ending Chat ...


### Tools

* Giving the Primary Agent the ability to compile fitness plans

In [12]:
import json
from agents import FunctionTool, RunContextWrapper, function_tool

@function_tool
def save_markdown_report(report: str, name:str):
    """Saves the full markdown report locally for a given fitness plan.
    Use this tool to save the full report once all the details are gathered.
    
    Args:
        report (str): Full markdown report
        name (str): User's name
    """
    
    try: 
        os.makedirs("reports", exist_ok=True)
        with open(f"./reports/{name}_report.md", "w") as f:
            f.write(report)
            
        return "Report Saved succesfully"
            
    except Exception as e:
        print(e)
        return f"Error saving report: {e}"
            
        
    

In [13]:
primary_agent = Agent(
    name="Fitness Assistant",
    instructions="""
    You are a fitness assistant that helps understand a user, their fitness profile, 
    their fitness profile, and their preferences and goals to provide personalized
    recommendations and guidance
    """,
    model=MODEL,
    tools=[save_markdown_report]
)

In [16]:
chat = []
await start_chat(primary_agent, chat)

Chat Started
User: Hi
Assistant: Hello! How can I assist you with your fitness goals today?
User: Can you help me create a fitness plan
Assistant: Absolutely! To create a personalized fitness plan, I need to gather some information from you. Here are a few questions:

1. **What are your fitness goals?** (e.g., weight loss, muscle gain, endurance, overall health)
2. **What is your current fitness level?** (e.g., beginner, intermediate, advanced)
3. **How many days a week do you want to work out?**
4. **What types of exercises do you enjoy or prefer?** (e.g., cardio, strength training, yoga, sports)
5. **Do you have any injuries or health concerns?**
6. **What equipment do you have access to?** (e.g., gym, home equipment, none)

Feel free to answer any or all of these questions!
User: My main goal is weight loss, I am a beginner. I can work out 4 days a week, I like strength training. I have asthma. I have access to a gym.
Assistant: Great! Thanks for sharing that information. Based on y